<a href="https://colab.research.google.com/github/pgosar/AlphaHacks/blob/main/brand_embeddings/wiki_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fire
!pip install wikipedia

!pip install beautifulsoup4
!pip install google
!pip install requests

     |████████████████████████████████| 92kB 10.3MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=dbe8bcbdaf14c735fd8c16f6e6477d25bb1fc21cc6998831dae808fd90c6c04b
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=05afd912f361f6ded202caabe2e0897f7e29017abc7dfef1f1a7bb3b971e8c11
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [ ]:
import io
import os
import shutil
import re
import string
import tensorflow as tf
import numpy as np

import logging
import wikipedia
import random

from bs4 import BeautifulSoup as bs
import requests
import json

import nltk
from nltk.corpus import stopwords

from googlesearch import search

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#NEW DATASET


In [ ]:

brand_list_link = "https://en.wikipedia.org/wiki/List_of_brand_name_food_products"

link_title = brand_list_link.split('/')[-1]  #Isolate title of wiki page

#API Request to get json data for link
api_request = "https://en.wikipedia.org/w/api.php?action=query&format=json&titles=" + link_title

#API Request to get all links in form of json data for link
links_api_request = "https://en.wikipedia.org/w/api.php?action=query&prop=links&format=json&titles=" + link_title

#Make API Calls
response = requests.get(api_request)
links_response = requests.get(links_api_request)
#print(response.json())
print(links_response.json()['query']['pages']['9091733']['links'])

[{'ns': 0, 'title': '5-4-3-2-1'}, {'ns': 0, 'title': 'Act II (popcorn)'}, {'ns': 0, 'title': 'Almarai'}, {'ns': 0, 'title': 'Alpro'}, {'ns': 0, 'title': 'Ambrosia (food brand)'}, {'ns': 0, 'title': 'Amul'}, {'ns': 0, 'title': "Amy's Kitchen"}, {'ns': 0, 'title': "Andy Capp's fries"}, {'ns': 0, 'title': 'Angel Delight'}, {'ns': 0, 'title': 'Apple Chips'}]


In [ ]:

title_api = "https://en.wikipedia.org/w/api.php?action=query&format=json&titles="
search_api = "https://en.wikipedia.org/w/api.php?action=opensearch&limit=1&namespace=0&format=json&search="
pageid_api = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts|revisions&rvslots=*&rvprop=content&format=json&pageids="
summary_api = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exsentences=10&exlimit=1&explaintext=1&format=json&pageids="
summary_api_title = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exsentences=10&exlimit=1&explaintext=1&format=json&titles="
page_content_api = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts|revisions&exlimit=1&explaintext=1&format=json&pageids="

brand_list_fp = "/content/cleaned_brand_list.json"
emb_save_fp = "embeddings.json"

keyword_list = ['brand', 'company', 'business', 'retail', 'manufact']

In [ ]:
#GET LIST OF PAGE ID's
company_list_link = "https://en.wikipedia.org/wiki/List_of_food_companies"

res = requests.get(company_list_link)
soup = bs(res.text, "html.parser")
link_list = {}
page_dict = {}
for link in soup.find_all("a"):
    url = link.get("href", "")
    if "/wiki/" in url:
      link_list[link.text.strip()] = url

print(link_list)

for link_title in link_list:
  api_request = title_api + link_title
  response = requests.get(api_request)
  json_data = response.json()
  try:
    temp_value = list((json_data['query']['pages'].keys()))[0]
    print(temp_value)
    page_dict[link_title] = temp_value
  except KeyError:
    print("KeyError w/ ", link_title)
    continue

print(page_dict)

{'': '/wiki/Main_Page', 'dynamic list': '/wiki/Wikipedia:WikiProject_Lists#Incomplete_lists', 'reliable sources': '/wiki/Wikipedia:Reliable_sources', 'food production': '/wiki/Food_production', 'All Joy Foods': '/wiki/All_Joy_Foods', 'Bakers': '/wiki/Bakers_(bakery)', 'BOS Ice Tea': '/wiki/BOS_Ice_Tea', 'Cevital': '/wiki/Cevital', 'Choppies': '/wiki/Choppies', 'Clover': '/wiki/Clover_(dairy)', 'Colcom Foods': '/wiki/Colcom_Foods', 'Distell Group Limited': '/wiki/Distell_Group_Limited', 'Famous Brands': '/wiki/Famous_Brands', 'Golden Web': '/wiki/Golden_Web_(company)', 'Kenya Wine Agencies Limited': '/wiki/Kenya_Wine_Agencies_Limited', 'Les Domaines Agricoles': '/wiki/Les_Domaines_Agricoles', 'Meat Corporation of Namibia': '/wiki/Meat_Corporation_of_Namibia', 'Melcom': '/wiki/Melcom', 'Nile Breweries Limited': '/wiki/Nile_Breweries_Limited', 'Pioneer Foods': '/wiki/Pioneer_Foods', 'SOMED': '/wiki/SOMED', 'Spur Corporation': '/wiki/Spur_Corporation', 'Tiger Brands': '/wiki/Tiger_Brands',

In [ ]:
def checkpoint_data(in_dict, outPath):
  with open(outPath, 'w') as fp:
    json.dump(in_dict, fp)

In [ ]:
def load_checkpoint(inPath):
  with open(inPath) as fp:
    temp_dict = json.load(fp)
  return temp_dict

In [ ]:
test_title = 'Lion Dairy & Drinks'
test_id = page_dict['Lion Dairy & Drinks']
new_title = test_title.replace('&', '%26')
if id == '-1':
  result = requests.get(summary_api_title + new_title)
  json_data = result.json()
  new_id = list(json_data['query']['pages'])[0]
  print(requests.get(search_api + test_title).json())

['Lion Dairy ', ['Lion Dairy & Drinks'], [''], ['https://en.wikipedia.org/wiki/Lion_Dairy_%26_Drinks']]


In [ ]:
for j in search(test_title, tld="co.in", num=5, stop=10, pause=2):
  if "wikipedia" in j:
    new_name = j.split('/')[-1]
    print(new_name)
    break
search_dict = {}

#for title, id in page_dict.copy().items():
#  if id == '-1':
#    for j in search(test_title, tld="co.in", num=5, stop=10, pause=2):
#      if "wikipedia" in j:
#        new_name = j.split('/')[-1]
#        print(new_name)
#        break

search_dict = {title: page_dict[title] for title in list(page_dict) if page_dict[title] == '-1'}
#for result in wikipedia.search(new_name):
#  temp_obj = wikipedia.page(result, auto_suggest = False)


Bega_Dairy_%26_Drinks


In [ ]:
search_dict = {title: page_dict[title] for title in list(page_dict) if page_dict[title] == '-1'}

In [ ]:
search_dict = safe_copy

In [ ]:
#CLEAN INVALID ID DATA
keyword_list = ['brand', 'company', 'business', 'retail', 'manufact']
popped_list =[]

for title, id in search_dict.copy().items():
  found_alt = False
  for j in search(title + "food company", tld="co.in", num=5, stop=10, pause=2):
    if "wikipedia" in j:
      #print(j)
      new_name = j.split('/')[-1]
      #print(new_name)
      found_alt = True
      break

  search_dict.pop(title)
  if found_alt:
    result = requests.get(summary_api_title + new_name)
    json_data = result.json()
    new_id = list((json_data['query']['pages'].keys()))[0]
    print("New Name: {}, New ID: {}".format(new_name, new_id))
    search_dict[new_name] = new_id

  page_dict.pop(title)

New Name: Bega_Dairy_%26_Drinks, New ID: 5689929
New Name: Dachan_Food_(Asia), New ID: 21702189
New Name: Debauve_%26_Gallais, New ID: 22217826
New Name: C._Hahne_M%C3%BChlenwerke_GmbH_%26_Co._KG, New ID: 7437329
New Name: Coppenrath_%26_Wiese, New ID: 38096369
New Name: Oishi_(company), New ID: 42400664
New Name: Sumol_%2B_Compal, New ID: 25596266
New Name: Korea_Tobacco_%26_Ginseng_Corporation, New ID: 2280662
New Name: Gunnar_Dafg%C3%A5rd_AB, New ID: 10277249
New Name: A._L._Simpkin_%26_Co._Ltd, New ID: 35084767
New Name: F._Duerr_%26_Sons, New ID: 33895868
New Name: R._Torre_%26_Company,_Inc., New ID: 1044007
New Name: Vanberg_%26_DeWulf, New ID: 35017651
New Name: Portal:Companies, New ID: 12042184
New Name: Portal:Companies, New ID: 12042184
New Name: List_of_food_companies, New ID: 9235440
New Name: List_of_oilfield_service_companies, New ID: 1254377
New Name: List_of_radio-controlled_model_aircraft_kit_manufacturers, New ID: 25579338
New Name: List_of_touch-solution_manufacture

In [ ]:
#Canon
for title, id in search_dict.copy().items():
  if "List" in title or id == '-1' or "wiki" in title.lower():
    popped_list.append(title)
    search_dict.pop(title)
    continue

In [ ]:
safe_copy = search_dict

In [ ]:
search_dict

{'A._L._Simpkin_%26_Co._Ltd': '35084767',
 'Bega_Dairy_%26_Drinks': '5689929',
 'C._Hahne_M%C3%BChlenwerke_GmbH_%26_Co._KG': '7437329',
 'Coppenrath_%26_Wiese': '38096369',
 'Dachan_Food_(Asia)': '21702189',
 'Debauve_%26_Gallais': '22217826',
 'F._Duerr_%26_Sons': '33895868',
 'Gunnar_Dafg%C3%A5rd_AB': '10277249',
 'Hereford': '49035',
 'Korea_Tobacco_%26_Ginseng_Corporation': '2280662',
 'Oishi_(company)': '42400664',
 'Portal:Companies': '12042184',
 'R._Torre_%26_Company,_Inc.': '1044007',
 'Sumol_%2B_Compal': '25596266',
 'Vanberg_%26_DeWulf': '35017651'}

In [ ]:
#Canon
for title, id in search_dict.items():
  page_dict[title] = id

In [ ]:
#SAVE CHECKPOINT
checkpoint_data(page_dict, 'clean_page_dict.json')

In [ ]:
#LOAD CHECKPOINT
page_dict = load_checkpoint('clean_page_dict.json')
len(page_dict)

In [ ]:
for title, id in page_dict.copy().items():
  if "List" in title or "wiki" in title.lower():
    page_dict.pop(title)

In [ ]:
checkpoint = page_dict

In [ ]:
page_dict = checkpoint

In [ ]:
#test_dict = {k: page_dict[k] for k in list(page_dict)[:30]}
deleted_dict = {}

for title, id in page_dict.copy().items():
  result = requests.get(summary_api + id)
  json_data = result.json()

  try:
    summary = json_data['query']['pages'][id]['extract']
    if summary.strip() == '' or not any(element in summary for element in keyword_list):
      page_dict.pop(title)
      deleted_dict[title] = id
  except KeyError as ke:
    page_dict.pop(title)
    delete_dict[title] = id

In [ ]:
deleted_dict

In [ ]:
checkpoint_data(page_dict, 'cleaned_brand_list.json')

In [ ]:
len(page_dict)

1122

#OLD STUFF


In [ ]:
  try:
    summary = json_data['query']['pages'][new_id]['extract']
    if not any(element in summary for element in keyword_list):
      search_dict.pop(title)
      popped_list.append(title)
      continue
  except KeyError as ke:
    search_dict.pop(title)
    popped_list.append(title)

In [ ]:
#CLEAN PAGE ID LIST

#Fix titles with id = -1 (invalid id)
for title, id in page_dict.copy().items():
  if id == '-1':
    result = requests.get(summary_api_title + title.replace('&', '%26'))
    #result = requests.get(search_api + title)
    json_data = result.json()
    new_id = list((json_data['query']['pages'].keys()))[0]
    new_title = title.replace('&', '%26')
    print("New title: ", new_title, ", New id: ", new_id)

    page_dict.pop(title)
    if new_id != '-1':
      page_dict[new_title] = new_id

New title:  Debauve %26 Gallais , New id:  22217826
New title:  C. Hahne Mühlenwerke GmbH %26 Co. KG , New id:  7437329
New title:  Coppenrath %26 Wiese , New id:  38096369
New title:  Liwayway Holdings Company (Oishi) , New id:  -1
New title:  Sumol + Compal , New id:  -1
New title:  Korea Tobacco %26 Ginseng Corporation , New id:  2280662
New title:  BEHER , New id:  -1
New title:  Gunnar Dafgård , New id:  -1
New title:  A. L. Simpkin %26 Co. Ltd , New id:  35084767
New title:  F. Duerr %26 Sons , New id:  33895868
New title:  R. Torre %26 Company, Inc. , New id:  1044007
New title:  Vanberg %26 DeWulf , New id:  35017651
New title:  Companies portal , New id:  -1
New title:  Lists portal , New id:  -1
New title:  Companies by industry , New id:  -1
New title:  Fitness wear , New id:  -1
New title:  Oilfield service , New id:  -1
New title:  Beer and breweries , New id:  -1
New title:  Vineyards and wineries , New id:  -1
New title:  Modern armament , New id:  -1
New title:  Radio-c

{'dynamic list': '50748425',
 'reliable sources': '12565170',
 'food production': '4360999',
 'All Joy Foods': '19115591',
 'Bakers': '7878833',
 'BOS Ice Tea': '41214596',
 'Cevital': '22588580',
 'Choppies': '37992758',
 'Clover': '44344',
 'Colcom Foods': '10033146',
 'Distell Group Limited': '44278067',
 'Famous Brands': '28336279',
 'Golden Web': '16908120',
 'Kenya Wine Agencies Limited': '45384964',
 'Les Domaines Agricoles': '41259496',
 'Meat Corporation of Namibia': '37502642',
 'Melcom': '37578426',
 'Nile Breweries Limited': '43099810',
 'Pioneer Foods': '25111136',
 'SOMED': '41307847',
 'Spur Corporation': '52818335',
 'Tiger Brands': '25111081',
 'Tilda Uganda': '50831736',
 'Tongaat Hulett': '20979279',
 'Grupo Arcor': '1748038',
 'Havanna': '463746',
 'La Serenísima': '23521140',
 'Molinos Río de la Plata': '8680399',
 'SanCor': '8347308',
 'The a2 Milk Company': '48897238',
 'Baiada Poultry': '35484375',
 'Bakers Delight': '3090612',
 'Balfours': '11447284',
 'Baskin-

In [ ]:
#test_dict = {k: page_dict[k] for k in list(page_dict)[:30]}
test_dict = page_dict
deleted_dict = {}
keyword_list = ['brand', 'company', 'business', 'retail', 'manufact']
for title, id in test_dict.copy().items():
  result = requests.get(summary_api + id)
  json_data = result.json()
  try:
    summary = json_data['query']['pages'][id]['extract']
    if summary.strip() == '':
      deleted_dict[title] = id
    elif not any(element in summary for element in keyword_list):
      deleted_dict[title] = id
  except KeyError as ke:
    deleted_dict[title] = id

In [ ]:
test_str = "Lion Dairy & Drinks"
response = requests.get(summary_api_title + test_str.replace('&', '%26'))
json_data = response.json()
new_id = list((json_data['query']['pages'].keys()))[0]

new_id

'66669556'

In [ ]:
#FINAL FILTER
new_dict = deleted_dict

for title, id in new_dict.copy().items():
  result = requests.get(search_api + title)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(page_dict, orient = 'index')
len(df)

1508

In [ ]:
#Find only food companies

for title, id in page_dict.items():
  result = requests.get(summary_api_title + title.replace)

In [ ]:
wikipedia.page(pageid="24517189").url

'https://en.wikipedia.org/wiki/Kar%27s_Nuts'

#WORKING WITH 3k Dataset


In [ ]:
_brand_list_fpath = "/content/data/3k_brands.txt"

In [ ]:
DEFAULT_SET_BRANDS = set()
with open(_brand_list_fpath) as fp:
    for line in fp.readlines():
        line = line.strip()
        if not line:
            continue
        DEFAULT_SET_BRANDS.add(line)

set_brands = DEFAULT_SET_BRANDS

In [ ]:
#Google search code
def brand_search(brand_name):
  final_query = brand_name + "brand" + "company" + "wikipedia"
  j = ""
  for j in search(final_query, tld="co.in", num=3, start=0, stop=2, pause=2):
    if "wikipedia" in j:
      print(j)
      #return j
      #new_name = j.split('/')[-1]
      #break
brand_search("Boomer")

https://en.wikipedia.org/wiki/Wrigley_Company
https://en.wikipedia.org/wiki/Category:Wrigley_Company


In [ ]:
skipped_brands = []
disamb_brands = []
def get_wiki(brand_name):
  try:
    wiki_obj = wiki.page(brand_name, auto_suggest = false)
  except wikipedia.DisambiguationError as e:
    disamb_brands.append(brand_name)
  except wikipedia.PageError as pe:
    skipped_brands.append(brand_name)


In [ ]:
brand_list = [brand.encode('ascii', 'ignore').decode() for brand in set_brands]

obj_list = [wikipedia.page(brand_name, auto_suggest = False) for brand_name in brand_list]

PageError: ignored

In [ ]:
skipped_brands = []
disamb_brands = []
count = 0
for brand_name in set_brands:
  brand_name = brand_name.encode('ascii', 'ignore').decode()

  #print("New loop with {}".format(brand_name))
  #wiki_obj = wikipedia.page(brand_name, auto_suggest=False)

  try:
    wiki_obj = wikipedia.page(brand_name, auto_suggest=False)
  except wikipedia.DisambiguationError as e:
    #s = e.options[0]
    #print("EXCEPTION TRIGGERED, WITH: ", brand_name)
    #print("Options are: ", e.options)
    #print("EXCEPTION TRIGGERED, TRYING WITH: ", s)
    #wiki_obj = wikipedia.page(brand_name, auto_suggest=True)
    disamb_brands.append(brand_name)
    continue
  except wikipedia.PageError as pe:
    #print("Page error with ", brand_name)
    skipped_brands.append(brand_name)
    continue

  count += 1
  if count%100 is 0:
    print(count)
  #print("{brand_name}: {wiki_url}".format(brand_name=brand_name, wiki_url=wiki_obj.url))
  #text = wiki_obj.content

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


100


KeyboardInterrupt: ignored

In [ ]:
len(disamb_brands)

490

In [ ]:
"Toys âRâ Us".encode('ascii', 'ignore').decode()

'Toys R Us'

In [ ]:
len(skipped_brands) + len(disamb_brands)

966

In [ ]:
skipped_brands

['Fresh Steps',
 'Avexa Limited',
 'Lwenbru',
 'Coldwell Bank',
 'Po de Acar',
 'Safe Auto Insurance',
 'Seamless CMS',
 'Codan Limited',
 'Ypsilon S.A.',
 'Broadview Home Security',
 'Product Naming',
 'Resene',
 'Snappy Tom',
 'Hard Rock Caf',
 'Mary Holland Pty Limited',
 'Sticky Johnson',
 'Metrogames',
 'Fiber Plus',
 'Express Virtual Meetings',
 'WC SA',
 'Monsterpro MotorBikes',
 'Chocolates Garoto',
 'Kcel',
 "Eight O'Clock Coffee Company",
 'Quality Clotheslines',
 'Advantech',
 'Aerosoles',
 'Lifestyle Clotheslines',
 'Itasa',
 'Town House',
 "Watti's",
 'WisePower',
 'Original Ranch',
 'Global Txi Areo',
 'XO Interior Systems',
 'Nutrisse',
 'MEA Advisory',
 'Inco Rogers Communications Empire',
 'MECWA',
 'Divella',
 'Chunghwa Telecom Co',
 'Deborah Milano',
 'Biosilk',
 'Liqui-Gel',
 'Homedics',
 'Esprito Santo Financial Group',
 'Soft Scrub',
 'iLANs Integrated LAN Services',
 'Yum Yum!',
 'Lanege',
 "Gold'n Plump",
 'Acindar SA',
 'Mr. Tayto',
 'Gouden Carolus',
 'Total L

#Web Scraper test code

In [ ]:
!pip install beautifulsoup4
!pip install google

In [ ]:
brand_name = "boomer"
try:
  wiki_obj = wikipedia.page(brand_name, auto_suggest=False)
except wikipedia.DisambiguationError as e:
  print(wikipedia.search("Wrigley_Company")[0])

Wrigley Company


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
from googlesearch import search

brand_name = "Boomer"

final_query = brand_name + "brand" + "company" + "wikipedia"

for j in search(final_query, tld="co.in", num=5, stop=10, pause=2):
  if "wikipedia" in j:
    new_name = j.split('/')[-1]
    break

for result in wikipedia.search(new_name):
  temp_obj = wikipedia.page(result, auto_suggest = False)


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: ignored

In [ ]:
wikipedia.page(pageid="38400900").url

'https://en.wikipedia.org/wiki/Sargento'

In [ ]:
!pip install requests

In [ ]:
import requests

response = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Main%20Page")
new_response = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Wrigley_Company")
#print(response.json())
print(new_response.json())

{'batchcomplete': '', 'query': {'normalized': [{'from': 'Wrigley_Company', 'to': 'Wrigley Company'}], 'pages': {'463687': {'pageid': 463687, 'ns': 0, 'title': 'Wrigley Company'}}}}


In [ ]:
search(final)

In [ ]:
wikipedia.page("SpaceX").url

'https://en.wikipedia.org/wiki/Space'

In [ ]:
#TODO LIST
#Loop through each brand name
# try assigning wiki_obj
# except disambiguation error
#  run google query for brand name + brand + company
#  select first wikipedia article url
#  save url
#  get list of suggestions from disambiguation
#  Loop through suggestion list
#    if not another disambiguation
#     Create wiki.page object and get url
#     if wiki.page url == query url, break loop
# except Page not found error
#  add to page not found list